# Bla

[TensorFlow][] from Google is an open source software library for artificial neural
networks (esp. deep learning) and there is a lot of buzz around it right now.
Some find it [terrific][] while others seem to be quite [disappointed][]. I
think that although TensorFlow is not revolutionary as some say, it surely does
a lot of things right especially when it comes to the [interface and architecture][].
My verdict is that TensorFlow is here to stay and this should mean to every
data scientist that it's at least beneficial to become familiar with it.

In order to do so, my strategy is always the same. After having read some [basic tutorials][],
I pick myself a task that comprises some more advanced features of a library and
define a set of achievements I want to fulfill. In order to use TensorFlow as part
of a predictive application that could be deployed productively, I need it to be:

* comfortable to use similar to a regressor in [Scikit-Learn][],
* easy to debug (e.g. visualization of the graph, accessibility of variables etc.) and
* able to store a trained network for later usage.

The new part for me is that I now write, in the spirit of Einstein's "You do not really
understand something unless you can explain it to your grandmother.", an article
about my learnings and experiences.

There are already a lot of neural network examples For TensorFlow to find on the web.
Besides its own examples there is [TensorFlow-Examples][] and in terms of Scikit-Learn
usage [skflow][] is definitely noteworthy. Mainly to do something different, our
task will be to develop an old-school *radial basis function (RBF) network* with TensorFlow.
So let's start with the definition what that exactly is. An RBF network is an
artificial neural network consisting of three layers, input, output and a hidden
layer that uses radial basis functions as activation functions. An RBF is a
real-valued function whose value depend only on the radius, i.e. the distance to
the origin. So every function $$\phi$$ satisfying the property
$$\phi(\mathbf{x}) = \phi(\|\mathbf{x}\|)$$ is an RBF.

<img class="noZoom" src="/images/rbf-network.png" alt="Radial basis function network">


$$\varphi (\mathbf {x} )=\sum _{i=1}^{N}a_{i}\rho (||\mathbf {x} -\mathbf {c} _{i}||)$$

[TensorFlow]: https://www.tensorflow.org/
[terrific]: http://www.kdnuggets.com/2015/12/tensor-flow-terrific-deep-learning-library.html
[disappointed]: http://www.kdnuggets.com/2015/11/google-tensorflow-deep-learning-disappoints.html
[interface and architecture]: https://github.com/zer0n/deepframeworks/blob/master/README.md
[basic tutorials]: https://www.tensorflow.org/versions/master/tutorials/index.html
[Scikit-Learn]: http://scikit-learn.org/stable/
[California housing dataset]: http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html
[TensorFlow-Examples]: https://github.com/aymericdamien/TensorFlow-Examples
[skflow]: https://github.com/tensorflow/skflow

In [8]:
from __future__ import division, print_function, absolute_import

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import os.path
from datetime import datetime

import numpy as np
import pandas as pd
import pyprind

from sklearn.datasets import fetch_mldata
from sklearn.cluster import k_means
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(context='notebook', style='whitegrid', font_scale=2)
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import tensorflow as tf
tf.__version__

'0.8.0'

In [10]:
mnist = fetch_mldata('MNIST original')

In [11]:
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([ 0.,  0.,  0., ...,  9.,  9.,  9.])}

In [12]:
encoder = OneHotEncoder(sparse=False)
target = encoder.fit_transform(mnist.target[:, np.newaxis])
X_train, X_test, y_train, y_test = train_test_split(mnist.data / 255, target, test_size=0.2)

In [13]:
np.random.seed(0)
n_rbf = 64
n_samples = 10000
a = np.random.uniform(0, 1, size=(n_samples, n_rbf))
b = np.random.uniform(0, 1, size=(n_samples, n_rbf))
np.mean(np.sum((a - b)**2, axis=1))

10.677450092787716

dfdfadsfasd Text goes here

In [14]:
class ANN(object):
    def __init__(self, n_rbf=64):
        self.n_hidden = n_rbf
        self._graph = tf.Graph()
        self.session = None
        self._initialized = False

    def get_model(self, n_features, means):
        with tf.name_scope('input'):
            x = tf.placeholder("float", shape=[None, n_features], name='X')
        with tf.name_scope('hidden'):
            c = [tf.Variable(tf.constant(means[i,:].astype(np.float32), shape=[1, n_features]), name='c')
                 for i in range(self.n_hidden)]
            r = [tf.Variable([11.0], name="beta")
                 for i in range(self.n_hidden)]
            h = [tf.exp(-tf.div(tf.reduce_sum(tf.square(tf.sub(x, c[i])), 1, keep_dims=True), tf.square(r[i]))) 
                 for i in range(self.n_hidden)]           
            [tf.histogram_summary("c_{}".format(i), c) for i, c in enumerate(c)]
        with tf.name_scope('output'):
            a = tf.Variable(tf.truncated_normal([self.n_hidden, 10], stddev=0.1), name='A')
            y = tf.nn.softmax(tf.matmul(tf.concat(1, h), a), name='y')
            tf.histogram_summary("A", a)
            tf.histogram_summary("y", y)
        return y

    def add_cost(self, yhat):
        y = tf.placeholder("float", shape=[None, 10], name='Y')
        cross_entropy = -tf.reduce_sum(y * tf.log(yhat))
        tf.scalar_summary("loss", cross_entropy)
        return cross_entropy

    def _init(self, n_features, means):
        with self._graph.as_default():
            model = self.get_model(n_features, means)
            costfunc = self.add_cost(model)
            train_a = tf.train.GradientDescentOptimizer(0.01).minimize(
                costfunc, var_list=[self.get_var(u'output/A:0')], name='train_a')
            train_c = tf.train.GradientDescentOptimizer(0.01).minimize(
                costfunc, var_list=self.get_vars('hidden/c'), name='train_c')         
            self.session = tf.Session(graph=self._graph)
            self._writer = tf.train.SummaryWriter("./logs", self.session.graph_def)
            self._saver = tf.train.Saver()
            self.session.run(tf.initialize_all_variables())
            self._initialized = True
                
    def save(self, path):
        if not self._initialized:
            raise RuntimeError("Cannot save an uninitialized model")
        with self._graph.as_default():
            os.mkdir(path)
            self._saver.save(self.session, os.path.join(path, "checkpoint.data"))
        return self
        
    def restore(self, path):
        if self._initialized:
            self.session.close()
        self._graph = tf.Graph()
        self.session = tf.Session(graph=self._graph)
        with self._graph.as_default():
            self._saver = tf.train.import_meta_graph(os.path.join(path, "checkpoint.data.meta"))
            self._saver.restore(self.session, os.path.join(path, "checkpoint.data"))
        return self
    
    def get_var(self, name):
        return self.get_all_vars()[name]
    
    def get_vars(self, name):
        return [v for k, v in self.get_all_vars().items() if k.startswith(name)]
        
    def get_op(self, name):
        return self._graph.get_operation_by_name(name)
    
    def get_tensor(self, name):
        return self._graph.get_tensor_by_name(name)
    
    def get_all_vars(self):
        with self._graph.as_default():
            return {var.name: var for var in tf.all_variables()}
                
    def fit(self, X, y, steps=50, batch=200):
        n_samples, n_features = X.shape

        if not self._initialized:
            means = k_means(X, self.n_hidden)[0]
            self._init(n_features, means)

        pbar = pyprind.ProgBar(steps)
        with self._graph.as_default():
            merged_summaries = tf.merge_all_summaries()
        for step in range(steps):
            for _ in range(1000):
                batch_indices = np.random.randint(n_samples, size=batch)
                self.session.run(self.get_op('train_a'), 
                                 feed_dict={self.get_tensor('input/X:0'): X[batch_indices],
                                            self.get_tensor('Y:0'): y[batch_indices]})
            for _ in range(1000):
                batch_indices = np.random.randint(n_samples, size=batch)
                self.session.run(self.get_op('train_c'), 
                                 feed_dict={self.get_tensor('input/X:0'): X[batch_indices],
                                            self.get_tensor('Y:0'): y[batch_indices]})
            summaries = self.session.run(merged_summaries,                                 
                                         feed_dict={self.get_tensor('input/X:0'): X[batch_indices],
                                                    self.get_tensor('Y:0'): y[batch_indices]})
            self._writer.add_summary(summaries, step)
            pbar.update()

        return self

    def predict(self, X):
        return self.session.run(self.get_tensor('output/y:0'), feed_dict={self.get_tensor('input/X:0'): X})

In [21]:
ann = ANN(n_rbf=1)
X_train.shape

(56000, 784)

In [16]:
for step in range(1):
    ann.fit(X_train, y_train, steps=1, batch=200)
    preds = ann.predict(X_train)
    print(np.unique(np.argmax(preds, axis=1)))
    print(np.sum(np.argmax(preds, axis=1) == np.argmax(y_train, axis=1))/preds.shape[0])

0%  100%
[#] | ETA: 00:00:00
Total time elapsed: 00:00:06


[1]
0.110928571429


In [18]:
ann.save('model_dir2')

In [19]:
del ann

In [22]:
ann.restore('model_dir2')
preds = ann.predict(X_train)
print(np.unique(np.argmax(preds, axis=1)))
print(np.sum(np.argmax(preds, axis=1) == np.argmax(y_train, axis=1))/preds.shape[0])

[1]
0.110928571429


In [23]:
for i in range(10):
    h = ann.h[i].eval(session=ann.session, feed_dict={ann._x: X_train[47820,:][np.newaxis], ann._y: y_train[0,:][np.newaxis]})
    print(h)

AttributeError: 'ANN' object has no attribute 'h'